In [1]:
# =====================================================
# AIRBNB LISTINGS DATA ANALYSIS
# =====================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# =====================================================
# 1. LOAD AND EXPLORE DATA
# =====================================================

data = pd.read_csv("listings.csv")
data_original = data.copy()  # Keep original for comparison

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Shape: {data.shape[0]} rows × {data.shape[1]} columns\n")
print(data.head())

DATASET OVERVIEW
Shape: 10480 rows × 18 columns

      id                                               name  host_id  \
0  27886  Romantic, stylish B&B houseboat in canal district    97647   
1  28871                            Comfortable double room   124245   
2  29051                   Comfortable single / double room   124245   
3  44391    Quiet 2-bedroom Amsterdam city centre apartment   194779   
4  48373                Cozy family home in Amsterdam South   220434   

       host_name  neighbourhood_group           neighbourhood   latitude  \
0           Flip                  NaN            Centrum-West  52.387610   
1          Edwin                  NaN            Centrum-West  52.367750   
2          Edwin                  NaN            Centrum-Oost  52.365840   
3            Jan                  NaN            Centrum-Oost  52.371680   
4  Vesna & Misha                  NaN  Buitenveldert - Zuidas  52.327808   

   longitude        room_type  price  minimum_nights  number_

In [2]:
# =====================================================
# 2. DATA INFORMATION
# =====================================================

print("\n" + "="*60)
print("DATA TYPES & INFO")
print("="*60)
data.info()

print("\n" + "="*60)
print("STATISTICAL SUMMARY")
print("="*60)
print(data.describe())


DATA TYPES & INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10480 entries, 0 to 10479
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              10480 non-null  int64  
 1   name                            10480 non-null  object 
 2   host_id                         10480 non-null  int64  
 3   host_name                       10477 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   10480 non-null  object 
 6   latitude                        10480 non-null  float64
 7   longitude                       10480 non-null  float64
 8   room_type                       10480 non-null  object 
 9   price                           5874 non-null   float64
 10  minimum_nights                  10480 non-null  int64  
 11  number_of_reviews               10480 non-null  int64  
 12  last_review  

In [3]:
# =====================================================
# 3. MISSING VALUES ANALYSIS
# =====================================================

def analyze_missing_values(df):
    """Comprehensive missing values analysis"""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing_Count': missing.values,
        'Missing_Percentage': missing_pct.values,
        'Data_Type': df.dtypes.values
    })
    
    missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values(
        'Missing_Count', ascending=False
    )
    
    print("\n" + "="*60)
    print("MISSING VALUES SUMMARY")
    print("="*60)
    print(f"Total Rows: {len(df)}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Columns with Missing Values: {len(missing_df)}\n")
    
    if len(missing_df) > 0:
        print(missing_df.to_string(index=False))
    else:
        print("✓ No missing values found!")
    
    return missing_df

missing_before = analyze_missing_values(data)


MISSING VALUES SUMMARY
Total Rows: 10480
Total Columns: 18
Columns with Missing Values: 6

             Column  Missing_Count  Missing_Percentage Data_Type
neighbourhood_group          10480          100.000000   float64
              price           4606           43.950382   float64
        last_review           1097           10.467557    object
  reviews_per_month           1097           10.467557   float64
            license            119            1.135496    object
          host_name              3            0.028626    object


In [4]:
# =====================================================
# 4. MISSING VALUES HANDLING (BY DATA TYPE)
# =====================================================

print("\n" + "="*60)
print("HANDLING MISSING VALUES")
print("="*60)

# Remove completely empty column
if 'neighbourhood_group' in data.columns:
    data = data.drop(columns=['neighbourhood_group'])
    print("✓ Dropped 'neighbourhood_group' (100% missing)")

# NUMERIC COLUMNS - Fill with median grouped by category
if data['price'].isnull().any():
    data['price'] = data.groupby('room_type')['price'].transform(
        lambda x: x.fillna(x.median())
    )
    print("✓ Filled 'price' with median by room_type")

if data['reviews_per_month'].isnull().any():
    data['reviews_per_month'] = data['reviews_per_month'].fillna(0)
    print("✓ Filled 'reviews_per_month' with 0 (no reviews)")

# DATETIME COLUMNS - Fill with mode
if data['last_review'].isnull().any():
    data['last_review'] = pd.to_datetime(data['last_review'], errors='coerce')
    mode_date = data['last_review'].mode()[0]
    data['last_review'] = data['last_review'].fillna(mode_date)
    print(f"✓ Filled 'last_review' with mode date: {mode_date.date()}")

# CATEGORICAL/TEXT COLUMNS - Fill with 'Unknown'
if data['license'].isnull().any():
    data['license'] = data['license'].fillna('Unknown')
    print("✓ Filled 'license' with 'Unknown'")

if data['host_name'].isnull().any():
    data['host_name'] = data['host_name'].fillna('Unknown')
    print("✓ Filled 'host_name' with 'Unknown'")

# Verify no missing values remain
print("\n" + "="*60)
print("AFTER HANDLING")
print("="*60)
missing_after = analyze_missing_values(data)


HANDLING MISSING VALUES
✓ Dropped 'neighbourhood_group' (100% missing)
✓ Filled 'price' with median by room_type
✓ Filled 'reviews_per_month' with 0 (no reviews)
✓ Filled 'last_review' with mode date: 2025-09-07
✓ Filled 'license' with 'Unknown'
✓ Filled 'host_name' with 'Unknown'

AFTER HANDLING

MISSING VALUES SUMMARY
Total Rows: 10480
Total Columns: 17
Columns with Missing Values: 0

✓ No missing values found!


In [5]:
# =====================================================
# 5. VISUALIZATION: MISSING VALUES
# =====================================================

def plot_missing_values(df_before, title="Missing Values Analysis"):
    """Create comprehensive missing values visualization"""
    
    if len(df_before) == 0:
        print("✓ No missing values to visualize!")
        return
    
    fig, (ax1, ax2) = plt.subplots(
        1, 2, figsize=(16, max(6, len(df_before) * 0.5))
    )
    
    # Color coding
    colors = [
        '#e74c3c' if x > 50 else '#f39c12' if x > 20 else '#3498db'
        for x in df_before['Missing_Percentage']
    ]
    
    # Bar chart
    bars = ax1.barh(
        df_before['Column'],
        df_before['Missing_Percentage'],
        color=colors,
        edgecolor='black',
        linewidth=1.2
    )
    
    ax1.set_xlabel('Missing Percentage (%)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Columns', fontsize=12, fontweight='bold')
    ax1.set_title('Missing Values by Column', fontsize=14, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3, linestyle='--')
    ax1.set_xlim(0, 100)
    
    # Add percentage labels
    for bar, pct in zip(bars, df_before['Missing_Percentage']):
        ax1.text(
            bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
            f'{pct:.2f}%', ha='left', va='center',
            fontsize=9, fontweight='bold'
        )
    
    # Table
    ax2.axis('off')
    table_data = [
        [row['Column'], f"{int(row['Missing_Count'])}", 
         f"{row['Missing_Percentage']:.2f}%", str(row['Data_Type'])]
        for _, row in df_before.iterrows()
    ]
    
    table = ax2.table(
        cellText=table_data,
        colLabels=['Column', 'Count', 'Missing %', 'Type'],
        cellLoc='left',
        loc='center',
        bbox=[0, 0, 1, 1]
    )
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)
    
    # Style header
    for i in range(4):
        table[(0, i)].set_facecolor('#2c3e50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Style rows
    for i in range(1, len(table_data) + 1):
        pct = float(table_data[i-1][2].strip('%'))
        color = '#ffcccc' if pct > 50 else '#ffe6cc' if pct > 20 else '#cce6ff'
        for j in range(4):
            table[(i, j)].set_facecolor(color)
    
    ax2.set_title('Missing Values Details', fontsize=14, fontweight='bold')
    
    fig.suptitle(title, fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig('missing_values_analysis.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved: missing_values_analysis.png")
    plt.close()

plot_missing_values(missing_before, "Missing Values (Before Handling)")


✓ Saved: missing_values_analysis.png


In [6]:
# =====================================================
# 6. VISUALIZATION: DATA TYPES
# =====================================================

def plot_data_types(df):
    """Visualize data types distribution"""
    
    dtype_df = pd.DataFrame({
        "Column": df.columns,
        "Data_Type": df.dtypes.astype(str)
    })
    
    dtype_counts = dtype_df["Data_Type"].value_counts()
    
    print("\n" + "="*60)
    print("DATA TYPES SUMMARY")
    print("="*60)
    print(dtype_df.to_string(index=False))
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, max(8, len(dtype_df) * 0.4)))
    
    # Table
    ax1.axis('off')
    table = ax1.table(
        cellText=dtype_df.values.tolist(),
        colLabels=["Column Name", "Data Type"],
        cellLoc='left',
        loc='center',
        bbox=[0, 0, 1, 1]
    )
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    
    # Style header
    for col in range(2):
        table[(0, col)].set_facecolor('#34495e')
        table[(0, col)].set_text_props(color='white', weight='bold')
    
    # Style rows
    for i in range(1, len(dtype_df) + 1):
        for j in range(2):
            table[(i, j)].set_facecolor('#ecf0f1')
    
    ax1.set_title("Column Data Types", fontsize=14, fontweight='bold', pad=20)
    
    # Bar chart
    colors_palette = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
    bars = ax2.bar(
        range(len(dtype_counts)),
        dtype_counts.values,
        color=colors_palette[:len(dtype_counts)],
        edgecolor='black',
        linewidth=1.2
    )
    
    ax2.set_xticks(range(len(dtype_counts)))
    ax2.set_xticklabels(dtype_counts.index, rotation=45, ha='right')
    ax2.set_title("Data Type Distribution", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Data Type", fontsize=12, fontweight='bold')
    ax2.set_ylabel("Count", fontsize=12, fontweight='bold')
    ax2.grid(axis='y', linestyle='--', alpha=0.4)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax2.text(
            bar.get_x() + bar.get_width()/2, height,
            int(height), ha='center', va='bottom',
            fontsize=10, fontweight='bold'
        )
    
    fig.suptitle(
        f"Data Types Analysis ({len(df.columns)} Columns)",
        fontsize=16, fontweight='bold', y=0.98
    )
    
    plt.tight_layout()
    plt.savefig('data_types_analysis.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved: data_types_analysis.png")
    plt.close()

plot_data_types(data)


DATA TYPES SUMMARY
                        Column      Data_Type
                            id          int64
                          name         object
                       host_id          int64
                     host_name         object
                 neighbourhood         object
                      latitude        float64
                     longitude        float64
                     room_type         object
                         price        float64
                minimum_nights          int64
             number_of_reviews          int64
                   last_review datetime64[ns]
             reviews_per_month        float64
calculated_host_listings_count          int64
              availability_365          int64
         number_of_reviews_ltm          int64
                       license         object

✓ Saved: data_types_analysis.png


In [9]:
# =====================================================
# 7. SAVE CLEANED DATA
# =====================================================

print("\n" + "="*60)
print("SAVING CLEANED DATA")
print("="*60)

# Save as CSV
data.to_csv('listings_cleaned.csv', index=False)
print("✓ Saved: listings_cleaned.csv")

# =====================================================
# 8. FINAL SUMMARY
# =====================================================

print("\n" + "="*60)
print("FINAL DATASET SUMMARY")
print("="*60)
print(f"Original Shape: {data_original.shape}")
print(f"Cleaned Shape: {data.shape}")
print(f"Columns Removed: {data_original.shape[1] - data.shape[1]}")
print(f"Missing Values Remaining: {data.isnull().sum().sum()}")
print("\n✓ Data cleaning completed successfully!")
print("="*60)


SAVING CLEANED DATA
✓ Saved: listings_cleaned.csv

FINAL DATASET SUMMARY
Original Shape: (10480, 18)
Cleaned Shape: (10480, 17)
Columns Removed: 1
Missing Values Remaining: 0

✓ Data cleaning completed successfully!
